# Analyse badlands stratigraphic output

If the stratigraphic structure is turned on in the XmL input file, **badlands** produces sedimentary layers Hdf5 files. The stratigraphic layers are defined on a regularly spaced grid and a layer is recorded at each layer time interval given by the user.

Here we show how we can visualise quickly the structure of the stratigraphic layer in an IPython notebook.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import glob
import numpy as np
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
import scipy.ndimage.filters as filters
from scipy.interpolate import RectBivariateSpline
from scipy.ndimage.filters import gaussian_filter

from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap

from scripts import stratalAnalyse_basin as strata

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

# Loading stratigraphic file

First we need to load the stratigraphic file. The files are located in the **h5/** folder in the simulation main output folder and are named using the following convention:
- `sed.time`T`.hdf5`

with T the display time index.

To load a file you will need to give the folder name and the number of processors used in your simulation.

For more information regarding the function uncomment the following line.

In [ ]:
# help(strata.stratalSection.__init__)

In [ ]:
folder = 'output/h5/'
strat = strata.stratalSection(folder)

Then we need to load a particular output time interval (this is the T parameter in the hdf5 file name convention).

**Note**

This number is not always the number of sedimentary layers for this particular time step as you could have chosen in the input file to have more than 1 sedimentary layer recorded by output interval!

In [ ]:
stepCounter = len(glob.glob1(folder,"tin.time*"))-1
print("Number of visualisation steps created: ",stepCounter)

In [ ]:
# help(strat.loadStratigraphy)

In [ ]:
strat.loadStratigraphy(stepCounter) 

# Building a cross-section

We then slice the stratigraphic mesh to visualise the sedimentary architecture along a given cross-section.

To create the cross-section you will need to provide:
+ the position of the segment in the simulation space _(xo,yo)_ and _(xm,ym)_, 
+ a gaussian filter value for smoothing (_gfilt_ a value of 0 can be used for non-smoothing) and 
+ the resolution of the cross-section (based on a number of points: _nbpts_). 

In [ ]:
# Build a cross-section along X axis in the center of the domain
x1 = np.amin(strat.x)
x2 = np.amax(strat.x)
y1 = np.amax(strat.x)/2
y2 = np.amax(strat.x)/2

# Interpolation parameters
nbpts = strat.nx
gfilt = 1

In [ ]:
# help(strat.buildSection)

In [ ]:
strat.buildSection(xo = x1, yo = y1, xm = x2, ym = y2, pts = nbpts, gfilter = gfilt)

# Stratal stacking pattern

We use **plotly** to visualise the vertival cross-section of stratal stacking apttern.

In [ ]:
#help(strata.viewSection)

In [ ]:
strata.viewSection(width = 800, height = 500, cs = strat, 
            dnlay = 5, rangeX=[8000, 22000], rangeY=[-120,50],
            linesize = 0.5, title='Stratal stacking pattern coloured by time')

# Sequence stratigraphy methods proposed in this notebook

New analytical methods are proposed in recent years on the interpretation of depositional systems. Here, we apply three methods, including 
+ **(i) the systems tracts method** in which the declaration of stratigraphic sequences is based on relative sea-level change; 
+ **(ii) the shoreline trajectory analysis** that defines different trajectory classes according to the migration of shoreline; 
+ **(iii) the accommodation sucession method** that focuses on the objective observation of depositional trends and then defines different sequence sets reponding to the competing ration between the change of accommodation and sediment supply.

# 1- Systems Tracts Method - based on relative sea-level

There are several models of systems tracts within depositional sequences, here we use the most simple one called the four systems tract model (figure is from [Helland-Hansen & Hampson (2009)](http://onlinelibrary.wiley.com/wol1/doi/10.1111/j.1365-2117.2009.00425.x/full)): <img src="images/SystemsTract_RSL.png" alt="Systems tract defined based on relative sea-level"  width="300" height="300" border="10" />
- highstand systems tract **HST** 
- falling-stage systems tract **FSST**
- lowstand systems tract **LST**
- transgressive systems tract **TST**

For each of these system tracts we define a given color. We use the 'colorlover' library [link](http://moderndata.plot.ly/color-scales-in-ipython-notebook/)

## Relative base-level

We first visualize the sea-level curve, and then extract **manually** the time intervals that bound different systems tracts, as showed in the above figure.

In [ ]:
time, Sealevel = strata.readSea('data/sealevel.csv') 
# There are 100 (=strat.nz) stratigraphic layers. We need to extract the sea level value of each stratigraphic layer.
dnlay = int(time.shape[0]/strat.nz)
Time = np.array(time[::dnlay])
sealevel = np.array(Sealevel[::dnlay])
timeStep = int(np.amax(time)/strat.nz) 

# Plot sea-level
strata.viewData(x0 = Time/timeStep, y0 = sealevel, width = 600, height = 400, linesize = 3, 
                color = '#6666FF',title='Sea-level curve',xlegend='display steps',ylegend='sea-level position')

We will assign 4 different colors based on relative sea-level change. To visualise the colors you can copy the html code below in the following [website](https://www.w3schools.com/cssref/tryit.asp?filename=trycss_color_rgba):

```html
<!DOCTYPE html><html>
  <head>
	<style>
      #p1 {background-color:rgba(213,139,214,0.8);}
      #p2 {background-color:rgba(215,171,117,0.8);}
      #p3 {background-color:rgba(39,123,70,0.8);}
      #p4 {background-color:rgba(82,128,233,0.8);}
    </style>
  </head>
  <body>
    <p>RGB colors with opacity for Systems-Tracts:</p>
    <p id="p1">HST</p>
    <p id="p2">FSST</p>
    <p id="p3">LST</p>
    <p id="p4">TST</p>
  </body>
</html>
```
Define different colors for different systems tracts

In [ ]:
cHST = 'rgba(213,139,214,0.8)' 
cFSST = 'rgba(215,171,117,0.8)' 
cLST = 'rgba(39,123,70,0.8)' 
cTST = 'rgba(82,128,233,0.8)' 

Specify the extent of every systems tracts according to the relative sea-level change

In [ ]:
FSST1 = np.array([0,25],dtype=int)
LST1 = np.array([25,32],dtype=int)
TST1 = np.array([32,45],dtype=int)
HST1 = np.array([45,50],dtype=int)
FSST2 = np.array([50,75],dtype=int)
LST2 = np.array([75,82],dtype=int)
TST2 = np.array([82,95],dtype=int)
HST2 = np.array([95,101],dtype=int)

Build the color list for each systems tract

In [ ]:
# Build the color list
STcolors = []
for k in range(FSST1[0],FSST1[1]):
    STcolors.append(cFSST)
for k in range(LST1[0],LST1[1]):
    STcolors.append(cLST)
for k in range(TST1[0],TST1[1]):
    STcolors.append(cTST)
for k in range(HST1[0],HST1[1]):
    STcolors.append(cHST)
for k in range(FSST2[0],FSST2[1]):
    STcolors.append(cFSST)
for k in range(LST2[0],LST2[1]):
    STcolors.append(cLST)
for k in range(TST2[0],TST2[1]):
    STcolors.append(cTST)
for k in range(HST2[0],HST2[1]):
    STcolors.append(cHST)

## Plotting the resulting systems tracts

In [ ]:
# help(strata.viewSectionST)

In [ ]:
strata.viewSectionST(width = 800, height = 500, cs=strat, colors=STcolors,
                   dnlay = 2, rangeX=[8000, 22000], rangeY=[-120,50], 
                   linesize=0.5, title='Systems tracts interpreted based on relative sea-level (RSL)')

# 2- Shoreline position / accomodation & sedimentation change

+ Before doing the interpretation using the two another methods, we need to calculate the shoreline positions (shoreline trajectory), the accommodation change ($\delta A$) and sedimentation change ($\delta S$) at shoreline positions through time. 

+ In order to calculate the shoreline trajectory, $\delta A$ and $\delta S$ at the shoreline, we need the information of the sedimentary layers when they deposited. In this example, **`100`** outputs are recorded (**`nbout`**). We use a loop to read the output every **`2`** timesteps (**`nstep`**). 

In [ ]:
nbout = stepCounter
nstep = 2

The `sed.time`T`.hdf5` file starts from time1. Therefore, the first file we load is T=1, then followed by T=2, 4, 6, 8, ..., 100

In [ ]:
strat_all = [strata.stratalSection(folder)]
strat_all[0].loadStratigraphy(nstep)
k = 1
for i in range(nstep,nbout+1,nstep):
    strat = strata.stratalSection(folder)
    strat_all.append(strat)
    strat_all[k].loadStratigraphy(i)
    k += 1

## Building cross-sections parameters

We then slice the stratigraphic mesh to visualise the sedimentary architecture along a given cross-section.

To create the cross-section you will need to provide the position of the segment in the simulation space _(xo,yo)_  and _(xm,ym)_ , a gaussian filter value for smoothing ( _gfilt_ a value of 0 can be used for non-smoothing) and the resolution of the cross-section (based on a number of points: _nbpts_ ). 

In [ ]:
# This will take a while...
npts = len(strat_all)
for i in range(0,npts):
    strat_all[i].buildSection(xo = x1, yo = y1, xm = x2, ym = y2, pts = nbpts, gfilter = gfilt)

# Generate shoreline trajectory

The shoreline position is where its elevation equals to the sea level. Sometimes we could not find the position whose elevation equals to the sea level, we then have to use the most close position as a proxy of shoreline position. In this situation, different cases have to use different rules. Generally, there are two models which are delta buildup and basin filling, as showed below:

|<img src="images/delta_buildup.png" alt="geometry" width="500" height="200"/>| <img src="images/basin_filling.png" alt="sea level" width="480" height="200"/> |
|:-:|:-:|
|Delta buildup example|Basin filling example|


**(a)** represents the case of **delta** buildup. In this case, deltas form at the margin area. The rule to calculate the shoreline position (1) for the left-hand side: shoreline_left = **np.amin**(np.where(elevation **>=** sealeve)[0]); (2) for right-hand side: shoreline_right = **np.amax**(np.where(elevation **>=** sealeve)[0]).

**(b)** represents the case of **basin** filling. In this case, basin forms at the center area. The rule to calculate the shoreline position (1) for the left-hand side: shoreline_left = **np.amin**(np.where(elevation **<=** sealeve)[0]); (2) for right-hand side: shoreline_right = **np.amax**(np.where(elevation **<=** sealeve)[0]).

If there is only one side that need to calculate the shoreline trajectory on the cross-section, you still need specify the model style ('delta' or 'basin') first. Then you only use the shoreline position at that side and ignore the result of the other side.

In addition to shoreline trajectory, the accommodation change ($\delta A$) and sediementaion change ($\delta S$) will also be calculated at the shoreline position.

In [ ]:
time, Sealevel = strata.readSea('data/sealevel.csv') 
# Extract the value of sea-level for each stratigraphic layer
dnlay = int(time.shape[0]/nbout*nstep)
Time = np.array(time[::dnlay])
sealevel = np.array(Sealevel[::dnlay])
sealevel[0] = Sealevel[1]
timeStep = int(np.amax(time)/nbout*nstep) 

# Plot sea-level
# strata.viewData(x0 = Time/timeStep, y0 = sealevel, width = 600, height = 400, linesize = 3, 
#                 color = '#6666FF',title='Sea-level curve',xlegend='display steps',ylegend='sea-level position')

In [ ]:
# help(strat.buildParameters)

In [ ]:
strat.buildParameters(npts = npts, strat_all = strat_all, sealevel = sealevel, gfilter = 0.1, style = 'basin')

# 3 - Shoreline trajectory method

Here we use a second method to interpret the depositional units based on the shifts of shoreline (from [Helland-Hansen & Hampson (2009)](http://onlinelibrary.wiley.com/wol1/doi/10.1111/j.1365-2117.2009.00425.x/full)):
<img src="images/SystemsTract_TC.png" alt="Systems tract defined based on shoreline trajectory"  width="400" height="400" border="10" />

- Transgressive trajectory class ** TTC **, when shoreline landward shift
- Descending regressive trajectory class ** DRTC **, when shoreline basinward shift accompanied by degradation
- Ascending regressive trajectory class ** ARTC **, when shoreline basinward shift accompanied by aggradation
- Stationary trajectory class **STC**, when the shoreline stands still

## Visualize shoreline trajectory and its gradient

When calculating the shoreline trajectory gradient, the direction of shoreline migration along the cross-section at different sides (left or right) of different model styles (delta or basin) responds to transgression or regression differently. For example, in the delta example, the increase of shoreline position (gradient > 0) represents transgression at the left side, but means regression at the right side. In the basin example, the increase of shoreline position (gradient > 0) represents regression at the left side, but means transgression at the right side. 

Therefore, in the delta example: 
- gval (gradient) = np.gradient(shoreline_left); 
- gval (gradient) = np.gradient(-shoreline_right). 

While for the basin example: 
- gval (gradient) = np.gradient(-shoreline_left); 
- gval (gradient) = np.gradient(shoreline_right).

In [ ]:
# left-hand side
xval = np.linspace(0,strat.nz,npts)
yval = strat_all[0].dist[strat.shoreID_l.astype(int)]
gval = np.gradient(-yval)

# View shoreline position through time
strata.viewData(x0 = xval, y0 = yval, width = 600, height = 400, linesize = 3, color = '#6666FF',
               title='Shoreline trajectory',xlegend='display steps',ylegend='shoreline position in metres')

# Define the gradient evolution
strata.viewData(x0 = xval, y0 = gval, width = 600, height = 400, linesize = 3, color = '#f4a142',
               title='Shoreline trajectory gradient',xlegend='display steps',ylegend='gradient shoreline position')

In [ ]:
# right-hand side
xval = np.linspace(0,strat.nz,npts)
yval_r = strat_all[0].dist[strat.shoreID_r.astype(int)]
gval_r = np.gradient(yval_r)

# View shoreline position through time
strata.viewData(x0 = xval, y0 = yval_r, width = 600, height = 400, linesize = 3, color = '#6666FF',
               title='Shoreline trajectory',xlegend='display steps',ylegend='shoreline position in metres')

# Define the gradient evolution
strata.viewData(x0 = xval, y0 = gval_r, width = 600, height = 400, linesize = 3, color = '#f4a142',
               title='Shoreline trajectory gradient',xlegend='display steps',ylegend='gradient shoreline position')

We will assign 4 different colors based on relative shoreline change. To visualise the colors you can copy the html code below in the following [website](https://www.w3schools.com/cssref/tryit.asp?filename=trycss_color_rgba):

```html
<!DOCTYPE html><html>
  <head>
	<style>
      #p1 {background-color:rgba(56,110,164,0.8);}
      #p2 {background-color:rgba(60,165,67,0.8);}
      #p3 {background-color:rgba(112,54,127,0.8);}
      #p4 {background-color:rgba(246,146,80,0.8);}
    </style>
  </head>
  <body>
    <p>RGB colors with opacity for Systems-Tracts:</p>
    <p id="p1">TC</p>
    <p id="p2">STC</p>
    <p id="p3">ARC</p>
    <p id="p4">DRC</p>
  </body>
</html>
```
Define different colors for different shoreline trajectory classes

In [ ]:
# Default color used: 
TC = 'rgba(56,110,164,0.8)'
STC = 'rgba(60,165,67,0.8)'  
ARC = 'rgba(112,54,127,0.8)'
DRC = 'rgba(252,149,7,0.8)'

# You can change them by specifying new values in the function
STcolors_ST = strata.build_shoreTrajectory(xval, yval, gval, sealevel, strat.nz, cTC=TC, cDRC=DRC, cARC=ARC, cSTC=STC)
# For a different side
STcolors_ST_r = strata.build_shoreTrajectory(xval, yval_r, gval_r, sealevel, strat.nz, cTC=TC, cDRC=DRC, cARC=ARC, cSTC=STC)

## Plotting the resulting shoreline trajectory classes

When the shoreline migrations on both sides share the same trend, STcolors_ST =  STcolors_ST_r. 

In [ ]:
strata.viewSectionST(width = 800, height = 500, cs = strat, colors = STcolors_ST, 
                     dnlay = 2, rangeX=[8000, 22000], rangeY=[-120,50], 
                     linesize = 0.5, title = 'Classes interpreted based on shoreline trajectory (ST)')

# 4 - Accommodation succession method

This accommodation succession method is referred to [Neal & Abreu (2009)]((https://www.researchgate.net/publication/249521744_Sequence_stratigraphy_hierarchy_and_the_accommodation_succession_method)), which is based on the relationship between the rate of changes in accommodation and sedimentation. 
<img src="images/SystemsTract_AS.png" alt="Systems tract defined based on shoreline trajectory"  width="600" height="400" border="10" />

- Retrogradation sequence set ** RSS **
- Aggradation to Progradation to Degradation (maybe) sequence set ** APDSS **
- Progradation to Aggradation sequence set ** PASS **

In [ ]:
# left-hand side
xval = np.linspace(0,strat.nz,npts)
ASval = strat.accom_l-strat.sed_l
gval = np.gradient(ASval)

# Accommodation (A) and sedimentation (S) change differences
strata.viewData(x0 = xval, y0 = ASval, width = 600, height = 400, linesize = 3, 
                color = '#6666FF',title='Change between accomodation & sedimentation',xlegend='display steps',
                ylegend='A-S')

# Define the gradient evolution
strata.viewData(x0 = xval, y0 = gval, width = 600, height = 400, linesize = 3, color = '#f4a142',
               title='A&S gradient',xlegend='display steps',ylegend='gradient A&S')

In [ ]:
# right-hand side
xval = np.linspace(0,strat.nz,npts)
ASval_r = strat.accom_r-strat.sed_r
gval_r = np.gradient(ASval_r)

# Accommodation (A) and sedimentation (S) change differences
strata.viewData(x0 = xval, y0 = ASval_r, width = 600, height = 400, linesize = 3, 
                color = '#6666FF',title='Change between accomodation & sedimentation',xlegend='display steps',
                ylegend='A-S')

# Define the gradient evolution
strata.viewData(x0 = xval, y0 = gval_r, width = 600, height = 400, linesize = 3, color = '#f4a142',
               title='A&S gradient',xlegend='display steps',ylegend='gradient A&S')

We will assign 3 different colors based on relative accommodation space and sedimentation rate. To visualise the colors you can copy the html code below in the following [website](https://www.w3schools.com/cssref/tryit.asp?filename=trycss_color_rgba):

```html
<!DOCTYPE html><html>
  <head>
	<style>
      #p1 {background-color:rgba(51,79,217,0.8);}
      #p2 {background-color:rgba(252,149,7,0.8);}
      #p3 {background-color:rgba(15,112,2,0.8);}
    </style>
  </head>
  <body>
    <p>RGB colors with opacity for Systems-Tracts:</p>
    <p id="p1">RSS</p>
    <p id="p2">PASS</p>
    <p id="p3">APDSS</p>
  </body>
</html>
```

Define different colors for different accommodation succession sequence sets.

In [ ]:
# Default color used: 
R = 'rgba(51,79,217,0.8)' 
APD = 'rgba(252,149,7,0.8)' 
PA= 'rgba(15,112,2,0.8)'

# You can change them by specifying new values in the function
STcolors_AS = strata.build_accomSuccession(xval,ASval,gval,strat.nz,cR=R,cPA=PA,cAPD=APD)
# For a different side
STcolors_AS_r = strata.build_accomSuccession(xval,ASval_r,gval_r,strat.nz,cR=R,cPA=PA,cAPD=APD)

## Plotting the resulting accommodation succession sequence sets

In [ ]:
strata.viewSectionST(width = 800, height = 500, cs = strat, colors = STcolors_AS, 
                     dnlay = 2, rangeX=[8000, 22000], rangeY=[-120,50], linesize = 0.5, 
                     title = 'Sequence sets interpreted based on change of accommodation and sedimentation (AS)')

# 5 - Depositional environments

Badlands outputs preserve the information (e.g. depth, thickness) of each stratigraphic layer when the layer is formed. By extracting these information, we can define different depositional environments (lithofacies) based on the paleo-depth.

We use different numbers (**enviID**) to represent different depositional environment. For example,

- **enviID = 1 -> alluvial plain**:   30 m above sea level
- **enviID = 2 -> shoreface**:        0 - 50 m below sea level
- **enviID = 3 -> slope**:            50 - 200 m below sea level
- **enviID = 4 -> deep marine**:      200 - 600 m below sea level
- **enviID = 5 -> ocean basin**:      > 600 m below sea level

In [ ]:
# Specify the range of water depth (relative to sea level, positive = below sea level) of each depositional environment
depthIDs = [-30, 0, 30, 50, 100] 

# Build enviID list
enviID = []
for i in range(len(strat_all)):  # len(cs): number of layers that are read
    nz = strat_all[i].nz-1
    nbpts = strat_all[i].dist.shape[0]  # nbpts: interpolate space
    for j in range(nbpts):  
        if ((strat_all[i].secDep[nz][j]) > (sealevel[i] - depthIDs[0])):
            enviID.append(0)
        elif ((strat_all[i].secDep[nz][j]) > (sealevel[i] - depthIDs[1])):
            enviID.append(1)  # alluvial plain
        elif ((strat_all[i].secDep[nz][j]) > (sealevel[i] - depthIDs[2])):
            enviID.append(2)  # shoreface
        elif ((strat_all[i].secDep[nz][j]) > (sealevel[i] - depthIDs[3])):
            enviID.append(3)  # slope
        elif ((strat_all[i].secDep[nz][j]) > (sealevel[i] - depthIDs[4])):
            enviID.append(4)  # deep marine
        else:
            enviID.append(5)  # ocean basin
# 
enviID = np.array(enviID)
enviID = np.reshape(enviID, (len(strat_all), nbpts))

## Wheeler Diagram

Wheeler diagram (or chronostratigraphic chart) is a powerful tool to document unconformities between sequences, and to understand the stacking patterns of sedimentary successions and their relationship to sea level. It displays the horizontal distribution of contemporaneous sedimentary layer sequences, as well as hiatuses in sedimentation. 

In [ ]:
# fig = plt.figure(figsize = (7,5))
fig, ax = plt.subplots(figsize=(7,6))
plt.rc("font", size=10)
# make a color map of fixed colors
color = ['white','limegreen','sandybrown','khaki','c','teal']
cmap = colors.ListedColormap(color)
bounds=[0,1,2,3,4,5,6]
norm = colors.BoundaryNorm(bounds, cmap.N)
img = plt.imshow(np.flip(enviID, 0), cmap=cmap, norm=norm, interpolation='nearest', extent=[0,30,0,1], aspect=20)
cbar = plt.colorbar(img, cmap=cmap, norm=norm, boundaries=bounds, fraction=0.046, pad=0.04)
cbar.ax.set_yticklabels(['-100','-50','-30','0','30','50','100'])
cbar.set_label('Paleo-depth (m)')
# 
for j in range(0,npts): 
        plt.axhline(Time[j]/1e6, color='k', linewidth=0.1)
# Plot shoreline trajectory
plt.scatter(strat.dist[strat.shoreID_l.astype(int)]/1000, Time/1e6, s=6, color='k')  # left-side
plt.scatter(strat.dist[strat.shoreID_r.astype(int)]/1000, Time/1e6, s=6, color='k')  # right-side

plt.xlim(7.5, 22.5)
plt.ylim(0, 1.0)

plt.xlabel('Distance (km)')
plt.ylabel('Time (Myr)')
plt.title('Wheeler Diagram')
# 
#fig.savefig("WheelerDiag.png", dpi=400)

# Vertical stacking pattern

Vertical stacking patterns (synthetic wells) could also be extracted at different positions to show the thickness of parasequences and the trend of depositional units through time.

To extract the vertical staking, first we need to calculate the stacked strata thickness at each position, then color the strata according to the water depth when they are deposited.

In [ ]:
# position of wells (km)
posit = np.array([11, 12, 18, 19, 20])  # position of wells /km
positID = [int(x*(nbpts-1)*1000/strat.x.max()) for x in posit]  # find the index of wells in strat.dist
color = ['white','limegreen','sandybrown','khaki','c','teal']
# Build color list for vertical stackings
color_fill = []
for i in positID:
    for j in range(0,strat.nz,nstep):
        if ((strat.secElev[j][i]) > (- depthIDs[0])):
            color_fill.append(color[0])
        elif (strat.secElev[j][i]) > (- depthIDs[1]):
            color_fill.append(color[1])
        elif (strat.secElev[j][i]) > (- depthIDs[2]):
            color_fill.append(color[2])
        elif (strat.secElev[j][i]) > (- depthIDs[3]):
            color_fill.append(color[3])
        elif (strat.secElev[j][i]) > (- depthIDs[4]):
            color_fill.append(color[4])
        else:
            color_fill.append(color[5])
colorFill = np.reshape(color_fill, (len(positID), int(strat.nz/nstep)+1))
# 
layth = []
for m in positID:
    nz = strat.nz-1
    layth.append(strat.secDep[nz][m])
    for n in range(1,int(strat.nz/nstep)):
        layth.append(-sum(strat.secTh[(nz-n*nstep):(nz-(n-1)*nstep)])[m])
layTh = np.reshape(layth, (len(positID), int(strat.nz/nstep)))

In [ ]:
import matplotlib as mpl
fig = plt.figure(figsize = (4,5))
plt.rc("font", size=10)
# 
ax = fig.add_axes([0.18,0.06,0.82,0.91])
data = layTh
bottom1 = np.cumsum(data[0], axis=0)
colors_1 = np.fliplr([colorFill[0]])[0]
plt.bar(0, data[0][0], color = 'w', edgecolor='lightgrey', hatch = '/')
for j in range(1, data[0].shape[0]):
    plt.bar(0, data[0][j], color=colors_1[j], edgecolor='black', bottom=bottom1[j-1])
# 
bottom2 = np.cumsum(data[1], axis=0)
colors_2 = np.fliplr([colorFill[1]])[0]
plt.bar(2, data[1][0], color = 'w', edgecolor='lightgrey', hatch = '/')
for j in range(1, data[1].shape[0]):
    plt.bar(2, data[1][j], color=colors_2[j], edgecolor='black', bottom=bottom2[j-1])
# 
bottom3 = np.cumsum(data[2], axis=0)
colors_3 = np.fliplr([colorFill[2]])[0]
plt.bar(4, data[2][0], color = 'w', edgecolor='lightgrey', hatch = '/')
for j in range(1, data[2].shape[0]):
    plt.bar(4, data[2][j], color=colors_3[j], edgecolor='black', bottom=bottom3[j-1])
#
bottom4 = np.cumsum(data[3], axis=0)
colors_4 = np.fliplr([colorFill[3]])[0]
plt.bar(6, data[3][0], color = 'w', edgecolor='lightgrey', hatch = '/')
for j in range(1, data[3].shape[0]):
    plt.bar(6, data[3][j], color=colors_4[j], edgecolor='black', bottom=bottom4[j-1])
#
bottom5 = np.cumsum(data[4], axis=0)
colors_5 = np.fliplr([colorFill[4]])[0]
plt.bar(8, data[4][0], color = 'w', edgecolor='lightgrey', hatch = '/')
for j in range(1, data[4].shape[0]):
    plt.bar(8, data[4][j], color=colors_5[j], edgecolor='black', bottom=bottom5[j-1])
#
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_color('none')
ax.axes.get_xaxis().set_visible(False)
ax.tick_params(axis='both', labelsize=10)
ax.yaxis.set_ticks_position('left')
# plt.xlim(-0.4,8)
# plt.ylim(8.5,6.5)
plt.ylabel('Elevation (m)',fontsize=10)
plt.yticks(fontsize=10)
# 
# fig.savefig("/workspace/volume/basin/VerticalStack.png", dpi=400)